In [1]:
from p2pfl.node import Node
from p2pfl.learning.learner import MyNodeLearning
from p2pfl.agregator import FedAvg    
from collections import OrderedDict
import torch
import time

def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(host="localhost")
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    return nodes

nodes = test_convergence((4,2))

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmp7hi9urys
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmp7hi9urys/_remote_module_non_sriptable.py
INFO:root:Nodo a la escucha en localhost 49627
INFO:root:Nodo a la escucha en localhost 49628
INFO:root:Nodo a la escucha en localhost 49629
INFO:root:Nodo a la escucha en localhost 49630
INFO:root:Conexión aceptada con ('localhost', 49628)
INFO:root:Conexión aceptada con ('localhost', 49629)
INFO:root:Conexión aceptada con ('localhost', 49627)
INFO:root:Conexión aceptada con ('localhost', 49630)
INFO:root:Conexión aceptada con ('localhost', 49628)
INFO:root:Conexión aceptada con ('localhost', 49627)
INFO:root:Broadcasting start learning...
INFO:root:Training...
INFO:root:Training...


GPU available: False, used: False


INFO:root:Training...


GPU available: False, used: False


INFO:root:Training...


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:root:Model added (2/4)
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (2/4)
INFO:root:Model added (2/4)
INFO:root:Model added (2/4)
INFO:root:Model added (3/4)
INFO:root:Broadcasting model to all clients...
DEBUG:root:Too mucho errors. ('localhost', 49629)
DEBUG:root:Last error: b'>=\xe6\xe5U9\x82\x18\x16\xbe\xdd\x17\xd8\xbdO\x98\x19\xbe;O"\xbdz\xe7\xd6=\xb4T\xca<\xa7\x10\xe1\xbd\n,n=\xe9m\x98=\x0c\x8ai=\x010v\xbdF+'
DEBUG:root:Closed connection with ('localhost', 49629)
INFO:root:Model added (3/4)
DEBUG:root:('localhost', 49627) (NodeConnection) Exception: [Errno 54] Connection reset by peer
INFO:root:Model added (3/4)
ERROR:root:[Errno 54] Connection reset by peer
Traceback (most recent call last):
  File "/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/node_connection.py", line 39, in run
    msg = self.socket.recv(BUFFER_SIZE)
ConnectionResetError: [Errno 54] Connection reset by peer
DEBUG:root:Closed connection with ('localhost', 49627)
INF

KeyboardInterrupt: 

In [2]:
for node in nodes:
        print(node.learner.get_parameters())
        print("\n\n\n\n")

NameError: name 'nodes' is not defined

In [1]:
while True:
    if nodes[1].round_models_shared:
        break
    time.sleep(0.1)

NameError: name 'nodes' is not defined